## knock 078 統計的に外れ値を除外する(IQR1.5倍)

In [6]:
import polars as pl
pl.Config.set_tbl_cols(-1)# 列の表示が省略されないようにする
import polars.selectors as cs# 抽出条件のプリセット

### データを読み込む

In [7]:
df_receipt = pl.read_csv("../docker/work/data/receipt.csv")
display(df_receipt.head())

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
i64,i64,str,i64,i64,str,str,i64,i64
20181103,1541203200,"""S14006""",112,1,"""CS006214000001…","""P070305012""",1,158
20181118,1542499200,"""S13008""",1132,2,"""CS008415000097…","""P070701017""",1,81
20170712,1499817600,"""S14028""",1102,1,"""CS028414000014…","""P060101005""",1,170
20190205,1549324800,"""S14042""",1132,1,"""ZZ000000000000…","""P050301001""",1,25
20180821,1534809600,"""S14025""",1102,2,"""CS025415000050…","""P060102007""",1,90


### ノック

In [8]:
# 売上高を計算する
df_group = (
    df_receipt
    # 非会員は除く
    .filter( ~pl.col("customer_id").str.starts_with("Z") )
    # 会員別の売上高を計算する
    .group_by(by = "customer_id")
    .agg(pl.sum("amount").alias("sum_amount"))
)

# 四分位数
pct25 = df_group["sum_amount"].quantile(quantile = 0.25)
pct75 = df_group["sum_amount"].quantile(quantile = 0.75)

# 上限と下限の計算
iqr = pct75 - pct25
usl = pct75 + 1.5 * iqr
lsl = pct25 - 1.5 * iqr

(
    df_group
    # 箱ひげ図の統計量に基づく外れ値を抽出する
    .filter(
        (pl.col("sum_amount") > usl)
        |
        (pl.col("sum_amount") < lsl)
    )
    # 先頭10行
    .head(n = 10)
)

customer_id,sum_amount
str,i64
"""CS007615000063…",11038
"""CS014214000023…",8405
"""CS028415000007…",19127
"""CS034515000173…",10074
"""CS006415000105…",10042
"""CS022515000028…",13512
"""CS012415000043…",14363
"""CS029415000194…",8550
"""CS007514000103…",9033
